In [ ]:
# ! pip install -U mlem scikit-learn fastapi uvicorn pandas numpy docker

## Training model

In [ ]:
from sklearn.datasets import load_iris

In [ ]:
data, y = load_iris(return_X_y=True, as_frame=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
        n_jobs=2,
        random_state=42,
)
rf.fit(data, y)

## Saving model

In [ ]:
from mlem.api import save

In [ ]:
model = save(
        rf,
        "rf",
        sample_data=data,
        description="Random Forest Classifier",
)

In [ ]:
! ls -la .

In [ ]:
! cat rf.mlem

In [ ]:
model.dict()

In [ ]:
import pickle

with open("rf", "rb") as f:
    rf2 = pickle.load(f)

rf2

In [ ]:
model.requirements.to_pip()

In [ ]:
with model.artifacts["data"].open() as f:
    rf3 = pickle.load(f)

rf3

In [ ]:
model.model_type

In [ ]:
model.model_type.dict()

In [ ]:
model.model_type.model

In [ ]:
model.model_type.io

In [ ]:
model.model_type.methods["predict"]

In [ ]:
model.model_type.methods["predict"].args[0].type_.columns

In [ ]:
model.model_type.methods["predict"].returns

## Saving data

In [ ]:
dataset = save(
        data,
        "iris.csv",
        description="Iris Dataset",
)

In [ ]:
dataset.dict()

In [ ]:
dataset.requirements.to_pip()

In [ ]:
dataset.reader.dict()

In [ ]:
dataset.data

In [ ]:
import pandas as pd

data2 = pd.read_csv("iris.csv")

In [ ]:
(data2 == data).all()

## MLEM Project

In [ ]:
! rm -rf rf* data.csv*

In [ ]:
! mlem init

In [ ]:
! ls -la .mlem

In [ ]:
! mlem config set pandas.default_format parquet

In [ ]:
save(
        rf,
        "rf",
        sample_data=data,
        description="Random Forest Classifier",
);
save(
        data,
        "iris",
        description="Iris Dataset",
);

In [ ]:
! mlem config get pandas.default_format

In [ ]:
! ls -la .mlem/data

In [ ]:
pd.read_parquet(".mlem/data/iris")

## MLEM CLI & API

In [ ]:
! mlem -h

In [ ]:
! mlem ls

In [ ]:
from mlem.api import __all__

__all__

## Loading

In [ ]:
from mlem.api import load, load_meta

load("rf")

In [ ]:
remote_rf = load_meta("rf", project="https://github.com/iterative/example-mlem-get-started", rev="simple")

In [ ]:
remote_rf.loc

In [ ]:
remote_rf.dict()

In [ ]:
remote_rf.load_value()
remote_rf.get_value()

In [ ]:
remote_rf.predict(data)

In [ ]:
rf.predict(data)

## Serving

In [ ]:
from mlem.api import serve

In [ ]:
! mlem types server fastapi

In [ ]:
! mlem serve rf fastapi -c port=80

In [ ]:
from mlem.api import build

In [ ]:
! mlem types builder

In [ ]:
! mlem types builder docker_dir

In [ ]:
build("docker_dir", "rf", server="fastapi", target="build/docker")

In [ ]:
! ls -la build/docker/

In [ ]:
! cat build/docker/requirements.txt

In [ ]:
! cat build/docker/Dockerfile

In [ ]:
! cat build/docker/run.sh

In [ ]:
! mlem types builder whl

In [ ]:
build("whl", remote_rf, package_name="my_rf", target="build/whl")

In [ ]:
! ls -la build/whl/

In [ ]:
! pip install build/whl/my_rf-0.0.0-py3-none-any.whl --no-deps

In [ ]:
import my_rf

In [ ]:
my_rf.predict(data)

## Cloud deployment

In [ ]:
! mlem declare env heroku staging

In [ ]:
! rm -rf .mlem/deployment/service.mlem

In [ ]:
from mlem.api import deploy

In [ ]:
deploy("service", "rf", "staging", app_name="mlem-tutorial-notebook")

In [ ]:
service = load("service")

In [ ]:
client = service.state.get_client()

In [ ]:
client.predict(data)

## Cleanup

In [ ]:
stop;  # stop "Run All Cells"

In [ ]:
! heroku apps:destroy mlem-tutorial-notebook --confirm mlem-tutorial-notebook

In [ ]:
! rm -rf .mlem iris* rf* build/

In [ ]:
! ls -la

In [ ]:
! pip uninstall my_rf